In [1]:
import pandas as pd
import numpy as np

In [2]:
cities = ['New York', 'Jacksonville', 'Dallas', 'San Antonio', 'San Diego', 'Houston', 'Phoenix', 'Philadelphia', 'Los Angeles', 'Seattle']

In [3]:
# Load the air quality dataset
AirQuality = pd.read_csv('/Users/rileychisholm/Downloads/US_AQI.csv')

In [4]:
AirQuality

,Unnamed: 0,CBSA Code,Date,AQI,Category,Defining Parameter,Number of Sites Reporting,city_ascii,state_id,state_name,lat,lng,population,density,timezone
0,0,10140,2022-01-01,21,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
1,1,10140,2022-01-02,12,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
2,2,10140,2022-01-03,18,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
3,3,10140,2022-01-04,19,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
4,4,10140,2022-01-05,17,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5617320,5718366,49740,1980-12-27,52,Moderate,CO,1,Yuma,AZ,Arizona,32.5995,-114.5491,137612.0,311.0,America/Phoenix
5617321,5718367,49740,1980-12-28,52,Moderate,CO,1,Yuma,AZ,Arizona,32.5995,-114.5491,137612.0,311.0,America/Phoenix
5617322,5718368,49740,1980-12-29,24,Good,CO,1,Yuma,AZ,Arizona,32.5995,-114.5491,137612.0,311.0,America/Phoenix
5617323,5718369,49740,1980-12-30,14,Good,CO,1,Yuma,AZ,Arizona,32.5995,-114.5491,137612.0,311.0,America/Phoenix


In [5]:
# Drops unnecessary columns
columns_to_drop = ['Unnamed: 0', 'density', 'population', 'CBSA Code', 'Number of Sites Reporting', 'state_name', 'lat', 'lng', 'timezone']
AirQuality.drop(columns=columns_to_drop, inplace=True)


In [6]:
# Renames columns
AirQuality.rename(columns={
    'city_ascii': 'City',
    'Defining Parameter': 'Main Pollutant'
}, inplace=True)


In [7]:
# Filters the dataset to include only the specified cities
AirQ = AirQuality[AirQuality['City'].isin(cities)]

In [8]:
# Checks for missing values
missing_values = AirQ.isnull().sum()
print(missing_values)

Date              0
AQI               0
Category          0
Main Pollutant    0
City              0
state_id          0
dtype: int64


In [9]:
AirQ

,Date,AQI,Category,Main Pollutant,City,state_id
6001,2022-01-01,53,Moderate,PM2.5,Dallas,TX
6002,2022-01-02,34,Good,Ozone,Dallas,TX
6003,2022-01-03,38,Good,Ozone,Dallas,TX
6004,2022-01-04,41,Good,Ozone,Dallas,TX
6005,2022-01-05,41,Good,NO2,Dallas,TX
...,...,...,...,...,...,...
5603470,1980-12-27,52,Moderate,CO,Seattle,WA
5603471,1980-12-28,76,Moderate,CO,Seattle,WA
5603472,1980-12-29,97,Moderate,CO,Seattle,WA
5603473,1980-12-30,101,Unhealthy for Sensitive Groups,CO,Seattle,WA


In [10]:
AirQ.info()

<class 'pandas.core.frame.DataFrame'>
Index: 155729 entries, 6001 to 5603474
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Date            155729 non-null  object
 1   AQI             155729 non-null  int64 
 2   Category        155729 non-null  object
 3   Main Pollutant  155729 non-null  object
 4   City            155729 non-null  object
 5   state_id        155729 non-null  object
dtypes: int64(1), object(5)
memory usage: 8.3+ MB


In [11]:
# Converts the 'Date' column to datetime
AirQ['Date'] = pd.to_datetime(AirQ['Date'])

/var/folders/jc/598nxw3x6rxcvsd9jn6h8b3w0000gn/T/ipykernel_61580/411707385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AirQ['Date'] = pd.to_datetime(AirQ['Date'])


In [12]:
# Sets the 'Date' column as the index
AirQ.set_index('Date', inplace=True)

In [13]:
def mode(series):
    return series.mode()[0]

In [14]:
# Groups by City and resamples by month
monthly_data = AirQ.groupby('City').resample('M').agg({
    'AQI': 'mean',
    'Category': lambda x: x.mode()[0],  
    'Main Pollutant': lambda x: x.mode()[0],  
    'state_id': 'first' 
}).reset_index()

In [15]:
# Reformats 'Date' column
monthly_data['Date'] = monthly_data['Date'].dt.to_period('M').dt.to_timestamp()

In [16]:
monthly_data

,City,Date,AQI,Category,Main Pollutant,state_id
0,Dallas,1980-01-01,38.258065,Good,NO2,TX
1,Dallas,1980-02-01,53.344828,Good,Ozone,TX
2,Dallas,1980-03-01,73.548387,Moderate,Ozone,TX
3,Dallas,1980-04-01,99.533333,Unhealthy for Sensitive Groups,Ozone,TX
4,Dallas,1980-05-01,101.096774,Unhealthy for Sensitive Groups,Ozone,TX
...,...,...,...,...,...,...
5066,Seattle,2021-10-01,43.774194,Good,Ozone,WA
5067,Seattle,2021-11-01,43.966667,Good,PM2.5,WA
5068,Seattle,2021-12-01,50.096774,Good,PM2.5,WA
5069,Seattle,2022-01-01,58.870968,Moderate,PM2.5,WA


In [17]:
monthly_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5071 entries, 0 to 5070
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   City            5071 non-null   object        
 1   Date            5071 non-null   datetime64[ns]
 2   AQI             5071 non-null   float64       
 3   Category        5071 non-null   object        
 4   Main Pollutant  5071 non-null   object        
 5   state_id        5071 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 237.8+ KB


In [18]:
monthly_data['Category'].value_counts()

Category
Moderate                          2611
Good                              1930
Unhealthy for Sensitive Groups     298
Very Unhealthy                     119
Unhealthy                          112
Hazardous                            1
Name: count, dtype: int64

In [19]:
monthly_data['Main Pollutant'].value_counts()

Main Pollutant
Ozone    2611
PM2.5    1227
NO2       627
CO        501
PM10      105
Name: count, dtype: int64

In [20]:
# Save to a new CSV file
# monthly_data.to_csv('/Users/rileychisholm/Downloads/US_AQI_monthly.csv', index=False)